In [5]:
#import all packages
import pygame
import os
import csv
from tkinter import *
import tkinter.messagebox
from tkinter import filedialog
from mutagen.mp3 import MP3
import time
import threading
from tkinter import ttk
from ttkthemes import themed_tk as tk
import json

#creating main window
root = tk.ThemedTk()
root.get_themes()
root.set_theme('arc') 

#root window contains status bar, left frame, right frame
#left frame contains playlist, add/del buttons
#right frame contains three more frames i.e. top, middle, bottom

#status bar
statusBar = ttk.Label(root, text='Welcome!', relief=SUNKEN, anchor='w', font='Times 15 italic')
statusBar.pack(side=BOTTOM, fill='x')


#create menubar
menuBar = Menu(root)
root.config(menu=menuBar)

"""Creating Frames"""
#creating right frame
rightFrame = Frame(root)
rightFrame.pack(side=RIGHT, padx=10)

#creating left frame
leftFrame = Frame(root)
leftFrame.pack(side=LEFT, padx=10)

#creating another left frame inside left frame
innerLeft = Frame(leftFrame)
innerLeft.pack(side=LEFT, padx=5)

#creating another right frame inside left frame
innerRight = Frame(leftFrame)
innerRight.pack(side=RIGHT, padx=5)

#creating right top frame
topFrame = Frame(rightFrame)
topFrame.pack(pady=30, padx=30)

#creating right middle frame
middleFrame = Frame(rightFrame)
middleFrame.pack(pady=15, padx=30)

#creating right bottom frame
bottomFrame = Frame(rightFrame)
bottomFrame.pack(pady=10)

#Creating Labels
timeLabel = ttk.Label(topFrame, text="Total Length : --:--", font='Ariel 12 bold')
timeLabel.pack()

lengthLabel = ttk.Label(topFrame, text="Remaining Length : --:--", relief=GROOVE, font='Ariel 12')
lengthLabel.pack(pady=5)

#creating listbox
scrollbar = Scrollbar(innerLeft, orient=HORIZONTAL)
songList = Listbox(innerLeft, xscrollcommand=scrollbar.set)
songList.pack(fill='x')
scrollbar.config(command=songList.xview)
scrollbar.pack(fill="x")
songList.config(xscrollcommand=scrollbar.set)

#active functions
playList = []
def show_details(playSong):    
    file_data = os.path.splitext(playSong)
    if file_data[1] == '.mp3': #if imported file has mp3 format then
        audio = MP3(playSong)
        totalLength = audio.info.length
        
    else: #if imported file has wav format then do
        song = pygame.mixer.Sound(playSong)
        totalLength = song.get_length()
    
    mins, secs = divmod(totalLength, 60)
    mins = round(mins)
    secs = round(secs)
    timeFormat = '{:2d}:{:2d}'.format(mins, secs)
    timeLabel['text'] = 'Total Length : '+' '+timeFormat
    t1 = threading.Thread(target=start_count, args=(totalLength, ))# """Running the threads i.e. multithreading"""
    t1.setDaemon(True)
    t1.start() # """Main thread"""
    
#Running the time length count
def start_count(t):
    global paused
    """pygame.mixer.music.get_busy() returns a false when the music is actively playing"""
    while t:
        if paused:
            continue
        else:
            mins, secs = divmod(t, 60)
            mins = round(mins)
            secs = round(secs)
            timeFormat = '{:2d}:{:2d}'.format(mins, secs)
            lengthLabel['text'] = 'Remaining Length : '+' '+timeFormat
            time.sleep(1)
            t -= 1
            if pygame.mixer.music.get_busy()==False:
                lengthLabel['text'] = 'Remaining Length : --:--'
                break
                
def play_music():
    global paused #initialize pause variable to False under mixer.init() for desired results

    time.sleep(1)
    if paused:
        paused = False
        pygame.mixer.music.unpause()
        statusBar['text'] = 'Music Resumed'    
    else:
        try:
            selectedSong = songList.curselection()
            selectedSong = int(selectedSong[0])
            playSong = playList[selectedSong]
            pygame.mixer.music.load(playSong)
            pygame.mixer.music.play()
            statusBar['text']= 'Music Playing'+' - '+os.path.basename(playSong)
            show_details(playSong)
        except Exception:
            tkinter.messagebox.showerror('File not found', "Media Player could'nt find the file, please check again..")
             
def stop_music():
    pygame.mixer.music.stop()
    statusBar['text']= 'Music Stopped'

def pause_music():
    #creating a global variable for the purpose of pause and resume
    global paused
    paused = True
    pygame.mixer.music.pause()
    statusBar['text']= 'Music Paused'    

def rewind_music():
    play_music()
    statusBar['text']= 'Music Rewinded'

def mute_music():
    global muted
    if muted: #if the volume is muted do
        muted = False
        pygame.mixer.music.set_volume(.7)
        scale.set(70)
        muteButton.configure(image=volumePhoto)    
    else: #if the volume is not muted do
        muted = True
        pygame.mixer.music.set_volume(0)
        scale.set(0)
        muteButton.configure(image=mutePhoto)        
        
def set_vol(val):
    volume = float(val)/100 #deviding the value of var/100 because volume scale inputs values from 0.00-1.00
    pygame.mixer.music.set_volume(volume)
    
def about_us():
    tkinter.messagebox.showinfo('Information about player', 'This Applications was created using tkinter')
    
def browse_file():
    global fileName
    fileName = filedialog.askopenfilename()    
    add_songs(fileName)

def add_songs(f): 
    global fileName
    f = os.path.basename(f)
    if f not in songList.get(0, 'end'):
        index = 0
        songList.insert(index, f)
        playList.insert(index, fileName) #playlist for actually playing the song
        index += 1
    else: pass
    
def del_song():
    selectedSong = songList.curselection()
    selectedSong = int(selectedSong[0])
    playList.pop(selectedSong)
    songList.delete(selectedSong)
    

def loading_list():
    directory = []
    with open('choices.csv', 'r', newline='') as f:
        reader =  csv.reader(f)
        for row in reader:
            directory.append(row)
    return directory
    
def add_playlist():
    var = myEnt.get()
    oldList = loading_list()
    if var not in oldList[0]:
        oldList[0].append(var)
        with open('choices.csv', 'w', newline='') as f:
            writer =  csv.writer(f)
            writer.writerow(oldList[0])
    else:
        pass
    
def rem_playlist():
    var = dirList.get()
    dirName = var + '.txt'
    os.remove(dirName)
    oldList = loading_list()
    oldList[0].remove(var)
    with open('choices.csv', 'w', newline='') as f:
        writer =  csv.writer(f)
        writer.writerow(oldList[0])
    clear_playlist()

def add_items():
    try:
        var = dirList.get()
        catagoryName = var + '.txt'
        cs = songList.curselection()[0]
        songName = songList.get(cs)
        try:
            with open(catagoryName) as f:
                catagory = json.load(f)
                if songName not in catagory['songName']:
                    catagory['songName'].append(songName) 
                    catagory['songDest'].append(playList[cs])            
        except: 
            catagory = {'songName': [],
                        'songDest': []}
            catagory['songName'].append(songName) 
            catagory['songDest'].append(playList[cs])

    
        with open(catagoryName, 'w') as f:
            json.dump(catagory, f)            
    except: pass
    
def del_items():
    try:
        var = dirList.get()
        catagoryName = var + '.txt'
        cs = songList.curselection()[0]
        with open(catagoryName) as f:
            catagory = json.load(f)
        del catagory['songName'][cs]
        del catagory['songDest'][cs]
        del_song()
        with open(catagoryName, 'w') as f:
            json.dump(catagory, f)            
    except:
        tkinter.messagebox.showwarning(title='Must Note!', message='Item or Playlist does not exist')

def show_playlist():
    try:
        clear_playlist()
        var = dirList.get()
        catagoryName = var + '.txt'
        with open(catagoryName) as f:
            catagory = json.load(f)
        for item in catagory['songName']:
            index = 0
            songList.insert(index, catagory['songName'][index])
            playList.insert(index, catagory['songDest'][index]) #playlist for actually playing the song
            index += 1
    except:
        tkinter.messagebox.showwarning(title='Must Note!', message='Playlist does not exist')
        
def clear_playlist():
    songList.delete(0, 'end')
    
def on_close():
    pygame.mixer.music.stop()
    root.destroy()
    

    
    
#create submenu and drop down menu
"""Creating File sub menu"""
subMenu = Menu(menuBar, tearoff=0)
menuBar.add_cascade(label='File', menu=subMenu)
subMenu.add_command(label='New File', command=browse_file)
subMenu.add_command(label='Exit', command=root.destroy)

"""Creating Help sub menu"""
subMenu = Menu(menuBar, tearoff=0)
menuBar.add_cascade(label='Help', menu=subMenu)
subMenu.add_command(label='About Us', command=about_us)

#Configuring Main Window
root.title('Media Player')
root.iconbitmap('headphones.ico')

#initializing pygame mixer i.e. to operate music command (play, stop, pause etc...)
pygame.mixer.init()
paused = False
muted = False

#Uploading all the required photos
playPhoto = PhotoImage(file="play.png")  
stopPhoto = PhotoImage(file="stop.png") 
pausePhoto = PhotoImage(file="pause.png")
rewindPhoto = PhotoImage(file="rewind.png")
mutePhoto = PhotoImage(file="mute.png")
volumePhoto = PhotoImage(file="volume.png")

#Initializing widgets   
"""play button"""
playButton = ttk.Button(middleFrame, image=playPhoto, command=play_music)
playButton.grid(row=0, column=0, padx=10)
"""stop button"""
stopButton = ttk.Button(middleFrame, image=stopPhoto, command=stop_music)
stopButton.grid(row=0, column=1, padx=10)
"""pause button"""
pauseButton = ttk.Button(middleFrame, image=pausePhoto, command=pause_music)
pauseButton.grid(row=0, column=2, padx=10)
"""rewind button"""
rewindButton = ttk.Button(bottomFrame, image=rewindPhoto, command=rewind_music)
rewindButton.grid(row=0, column=0, padx=10)
"""rewind button"""
muteButton = ttk.Button(bottomFrame, image=volumePhoto, command=mute_music)
muteButton.grid(row=0, column=1, padx=10)
"""add playlist items button"""
addButton = ttk.Button(innerRight, text='Add To Playlist', width=15, command=add_items)
addButton.grid(row=0, column=0, pady=5)
"""delete playlist items button"""
delButton = ttk.Button(innerRight, text='Del From Playlist', width=15, command=del_items)
delButton.grid(row=1, column=0, pady=5)
"""add playlist button"""
addButton = ttk.Button(innerRight, text='Add new Playlist', width=15, command=add_playlist)
addButton.grid(row=3, column=0, pady=5)
"""delete playlist button"""
delButton = ttk.Button(innerRight, text='Del Playlist', width=15, command=rem_playlist)
delButton.grid(row=2, column=0, pady=5)
""" Tkinter entry box """
myEnt = StringVar(root)
entry = Entry(innerRight, textvariable=myEnt)
entry.grid(row=4, column=0, pady=5, ipady=3)
"""creating Combobox"""
dirList = ttk.Combobox(innerLeft, values=loading_list()[0], postcommand=lambda: dirList.configure(values=loading_list()[0]))
dirList.pack(fill='x')
"""display playlist button"""
dspButton = ttk.Button(innerLeft, text='Display Playlist', command=show_playlist)
dspButton.pack(side=LEFT, padx=2)
"""clear playlist button"""
clrButton = ttk.Button(innerLeft, text='Clear Playlist', command=clear_playlist)
clrButton.pack(side=LEFT, padx=2)
"""volume scale"""
scale = ttk.Scale(bottomFrame, from_=0, to_=100, orient=HORIZONTAL, command=set_vol)
scale.set(70)
pygame.mixer.music.set_volume(0.7)
scale.grid(row=0, column=2, padx=10)


#starting the main loop of frames i.e. main window is displayed to the user because of frames
"""root.protocol is used to contact to some event"""
root.protocol('WM_DELETE_WINDOW', on_close) 
root.mainloop()
